In [29]:
import torch
import torch.nn as nn
from utils import get_data_loaders
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
from triplet_loss import * 
from tensorflow.keras.models import Sequential , load_model
import tensorflow as tf


## cearint model

In [2]:

class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        
        # CNN model 
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, padding=1)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2)
        
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, padding=1)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=2)
        
        self.conv3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
        self.relu3 = nn.ReLU()
        self.pool3 = nn.MaxPool2d(kernel_size=2)
        
        self.fc1   = nn.Linear(in_features=64*4*4, out_features=64)
        self.relu4 = nn.ReLU()
        
        #concatinat 
        self.fc2   = nn.Linear(in_features=64+256, out_features=128)
        self.relu5 = nn.ReLU()
        
        self.fc3   = nn.Linear(in_features=128 , out_features = 10)
        

        
    
    def forward(self, x ,features ):
        # Define the forward pass through the CNN
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.pool1(x)

        x = self.conv2(x)
        x = self.relu2(x)
        x = self.pool2(x)
        
        x = self.conv3(x)
        x = self.relu3(x)
        x = self.pool3(x)
        
        x = x.view(-1,64*4*4)
        x = self.fc1(x)
        x = self.relu4(x)
        # now x is the out put of cnn model
        x = torch.cat((x, features), dim=1)
        x = self.fc2(x)
        x = self.relu5(x)
        output = self.fc3(x)
        return output , x
    

   

In [3]:
#data loader
full_dataloaders, _ = get_data_loaders(
     filenames={
            'train': './data/12000_train_mnistmnistmsvhnsynusps.npz',
            'test': './data/12000_test_mnistmnistmsvhnsynusps.npz',
      },
       batch_size= 64
    )

datafiles to read:  {'train': './data/12000_train_mnistmnistmsvhnsynusps.npz', 'test': './data/12000_test_mnistmnistmsvhnsynusps.npz'}
reading ./data/12000_train_mnistmnistmsvhnsynusps.npz, number of samples: 60000
reading ./data/12000_test_mnistmnistmsvhnsynusps.npz, number of samples: 21600
reading ./data/12000_test_mnistmnistmsvhnsynusps.npz, number of samples: 21600


## training cnn model

In [15]:
#triplet loss coefficient
landa = 0
#creating model
model = Model()

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
t_loss    = triplet_loss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Train the model
for epoch in range(10):  # loop over the dataset multiple times
    print ( "epoch:",epoch)
    running_loss = 0.0
    for batch_indx, (images, features, domain_labels, digit_labels) in enumerate(full_dataloaders['train']):
        # get the inputs; data is a list of [inputs, labels]
        # inputs, labels = data
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs , x = model.forward(images,features)
        loss_crossEntopy = criterion(outputs, digit_labels)
        loss_triplet = t_loss.forward(x,domain_labels)
        loss = loss_crossEntopy + landa * loss_triplet
        # print(f"crossEntropy : {loss_crossEntopy} triplet loos {loss_triplet} loos: {loss}")

        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if batch_indx % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, batch_indx + 1, running_loss / 2000))
print ("end training")

epoch: 0
epoch: 1
epoch: 2
epoch: 3
epoch: 4
epoch: 5
epoch: 6
epoch: 7
epoch: 8
epoch: 9
end training


## save trained model

In [5]:
# save the model
torch.save(model.state_dict(), 'model.pt')

## test the accuracy 

In [35]:
# load the saved model
model_state_dict = torch.load('model.pt')

model = Model()
model.load_state_dict(model_state_dict)

<All keys matched successfully>

In [36]:
# Set the model to evaluation mode
model.eval()

# Initialize a variable to keep track of the total number of correct predictions
correct = 0

# Initialize a variable to keep track of the total number of images processed
total = 0

# Iterate over the test data in batches
with torch.no_grad():
    for batch_indx, (images, features, domain_labels, digit_labels) in enumerate(full_dataloaders['test']):
        # Forward pass
        outputs,_ = model(images,features)
        # Get the predicted labels
        _, predicted = torch.max(outputs.data, 1)
        # Update the total number of images processed
        total += digit_labels.size(0)
        # Update the total number of correct predictions
        correct += (predicted == digit_labels).sum().item()

# Calculate the accuracy
accuracy = 100 * correct / total

# Print the accuracy
print('Accuracy of the network on the test images: %d %%' % accuracy)

Accuracy of the network on the test images: 63 %


## part 5 of project